### Download Datasets from Google Drive

In [ ]:
%cd ~
!git clone https://github.com/mayocream/tschan

In [ ]:
%cd ~/tschan
%pip install gdown
%pip install --upgrade gdown

# acquire the dataset from http://www.manga109.org/ja/download.html
!gdown "***&confirm=t"
!unzip Manga109.zip
!mv Manga109_released_2021_12_30 datasets/manga109 # keep name consistent with scripts/convert_manga109_to_coco.py
!rm Manga109.zip

### Preprecessing datasets

In [ ]:
%cd ~/tschan/datasets

%pip install manga109api ultralytics
!python scripts/convert_manga109_to_coco.py \
    --manga109_root_dir manga109 \
    --dataset_version v2021.12.31 \
    --label_filename_prefix manga109_coco \
    --add_manga109_info

# convert coco to yolo format
!python scripts/convert_coco_to_yolo.py

### Training Yolov8

Create a config YAML file inside `datasets/manga109/yolo_labels/`

```
path: /root/tschan/datasets/manga109/yolo_labels  # dataset root dir
train: images/manga109_coco_90train
val: images/manga109_coco_4val
test: images/manga109_coco_15test

# Classes
names:
  0: frame
  1: text
```

In [ ]:
%cd ~/tschan/detection
%pip install ultralytics

from ultralytics import YOLO

# Load a model
model = YOLO('yolov8s.pt')  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data='../datasets/manga109/yolo_labels/manga109.yaml', epochs=100, verbose=True, batch=128)
# results = model.train(resume=True)
!cp -rv runs /notebooks/

### Inference

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('runs/detect/train/weights/best.pt')
# run inference on a single image
model.predict('../datasets/bluearchive_comics/images/101.jpg', save=True, imgsz=640, conf=0.5)

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('../yolov8n.pt')

# Export the model
model.export(format='onnx')